# TF - IDF (Term Frequency - Inverse Document Frequency)

## Goal of TF IDF
### Characterisistics of terms with high TF-IDF score:
- High frequency within a document (TF)
- Unique to few documents (IDF)

### Why is this useful?
This is useful from a search POV because it helps identify words that are important to a document. For example, if you're searching for a document about the python programming language, you'd expect the words "python" and "programming" to appear frequently. By calculating the TF-IDF score, we can identify words that are important to a document while filtering out common words like "the", "is", "and", etc. that appear in many documents.

## Term Frequency
This measures how often a word appears in a document. Words that appear more frequently in a document are likely important for that document.

## Inverse Document Frequency
Inverse Document Frequency (IDF): This measures how unique or rare a word is across the entire corpus. If a word appears in many documents, it's less helpful in distinguishing one document from another (e.g., "the"). On the other hand, if a word appears in just a few documents, it's more distinctive.

## Combining TF and IDF

$$ TF-IDF = TF * IDF $$

High TF-IDF: The word is frequent in the document but rare in the corpus (important and unique).

Low TF-IDF: The word is either common across the corpus or not frequent in the document (less important).



## Mathematical formula

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

### Term Frequency (TF):
$$
\text{TF}(t, d) = \frac{f_{t, d}}{\sum_{t' \in d} f_{t', d}}
$$

- $f_{t, d}$: The frequency of term $t$ in document $d$.
- $\sum_{t' \in d} f_{t', d}$: The total number of terms in document $d$.

### Inverse Document Frequency (IDF):
$$
\text{IDF}(t, D) = \log \left( \frac{|D|}{1 + |d \in D : t \in d|} \right)
$$

- $|D|$: The total number of documents in the collection $D$.
- $|d \in D : t \in d|$: The number of documents in which the term $t$ appears. 
- $1$: Added to avoid division by zero in case $t$ does not appear in any document.

---

### Explanation:
1. **Where $t$ is the term**:
   - A single word or token for which the TF-IDF score is being calculated.
2. **Where $d$ is the document**:
   - A specific document from the corpus $D$ where the term $t$ is being considered.
3. **Where $D$ is the corpus**:
   - The entire collection of documents.
4. **Where $f_{t, d}$ is the term frequency**:
   - The raw count of term $t$ in document $d$.
5. **Where $\log$ is the logarithm function**:
   - Used to scale down the effect of IDF when a term appears in many documents.

Let me know if you'd like further explanation or examples!


In [ ]:
import numpy as np
import pandas as pd

class TfidfVectorizer:

    def __init__(self):
        self.vocabulary = {}
        self.document_frequency = {}
        
    def fit(self, corpus): 
        for doc in corpus:
            for word in set(doc.lower().split()):
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)

        # doc frequency
        self.document_frequency = {term: 0 for term in self.vocabulary}
        for doc in corpus:
            unique_terms = set(doc.lower().split())
            for word in unique_terms:
                self.document_frequency[word] += 1

        # IDF calculation
        self.inverse_document_frequency = {}
        N = len(corpus)
        for term, df in self.document_frequency.items():
            self.inverse_document_frequency[term] = float(np.log(N/df+1))

    def transform(self, corpus):
        tf_idf_matrix = np.zeros((len(corpus), len(self.vocabulary)))

        for i, doc in enumerate(corpus):
            term_count = {}
            for term in doc.lower().split():
                term_count[term] = term_count.get(term, 0) + 1

            for term, count in term_count.items():
                if term in self.vocabulary:
                    tf = count
                    idf = self.inverse_document_frequency[term]
                    tf_idf_matrix[i][self.vocabulary[term]] = tf * idf

        return tf_idf_matrix 
    
    def fit_transform(self, corpus):
        self.fit(corpus)
        result = self.transform(corpus)
        return result

corpus = [
    "Apple Apple Apple",
    "Banana Mango Banana",
    "Cherry Cherry Cherry",
    "Grapes Grapes Berries Grapes",
    "Apple Banana Mango",
    "Blueberries Strawberries"
]

def format_matrix(vocab, matrix):
    if len(vocab) == len(matrix[0]):
        terms = list(vocab)
        return pd.DataFrame(
            data=matrix,
            columns=terms
        )
    else:
        raise ValueError("Vocabulary and Result matrix do not match")


tf_idf = TfidfVectorizer()
# tf_idf.fit(corpus)

result = tf_idf.fit_transform(corpus)

print(f"Vocab: {tf_idf.vocabulary}")
print(f"Document Frequency: {tf_idf.document_frequency}")
print(f"IDF: {tf_idf.inverse_document_frequency}")
print("Result:")
format_matrix(tf_idf.vocabulary, result)

Vocab: {'apple': 0, 'banana': 1, 'mango': 2, 'cherry': 3, 'grapes': 4, 'berries': 5, 'strawberries': 6, 'blueberries': 7}
Document Frequency: {'apple': 2, 'banana': 2, 'mango': 2, 'cherry': 1, 'grapes': 1, 'berries': 1, 'strawberries': 1, 'blueberries': 1}
IDF: {'apple': 1.3862943611198906, 'banana': 1.3862943611198906, 'mango': 1.3862943611198906, 'cherry': 1.9459101490553132, 'grapes': 1.9459101490553132, 'berries': 1.9459101490553132, 'strawberries': 1.9459101490553132, 'blueberries': 1.9459101490553132}
Result:


,apple,banana,mango,cherry,grapes,berries,strawberries,blueberries
0,4.158883,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.000000,2.772589,1.386294,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.000000,5.83773,0.00000,0.00000,0.00000,0.00000
3,0.000000,0.000000,0.000000,0.00000,5.83773,1.94591,0.00000,0.00000
4,1.386294,1.386294,1.386294,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,1.94591,1.94591
